Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 🍌


In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!



### Setup

In [1]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [0]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [3]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

C:\Users\thedo\instacart_2017_05_01


In [4]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

aisles.csv
departments.csv
orders.csv
order_products__prior.csv
order_products__train.csv
products.csv


### For each csv file, look at its shape & head 

In [13]:
import pandas as pd
from IPython.display import display

for filename in glob('*.csv'):
  df = pd.read_csv(filename)
  print('\n', filename, df.shape)
  display(df.head())


 aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation



 departments.csv (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol



 orders.csv (3421083, 7)


...

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0



 order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1



 products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [0]:
# Save all dataframes
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
orders = pd.read_csv('orders.csv')
order_products__prior = pd.read_csv('order_products__prior.csv')
order_products__train = pd.read_csv('order_products__train.csv')
products = pd.read_csv('products.csv')

## The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

## ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

# Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## What was the most frequently ordered product?

In [24]:
# Most re-ordered products
(order_products__train.groupby('product_id').sum().
 sort_values('reordered', ascending=False).head()['reordered'])

product_id
24852    16557
13176    13362
21137     8603
21903     8055
47766     6226
Name: reordered, dtype: int64

In [22]:
# Most ordered products
order_products__train['product_id'].value_counts().head()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
Name: product_id, dtype: int64

In [0]:
combined = pd.merge(order_products__train, products, on='product_id', how='inner')

In [41]:
combined['product_name'].value_counts()

Banana                                            18726
Bag of Organic Bananas                            15480
Organic Strawberries                              10894
Organic Baby Spinach                               9784
Large Lemon                                        8135
                                                  ...  
Peanut Butter Ice Cream Slice Bars                    1
41% Cacao Caramel With Sea Salt Milk Chocolate        1
Klassic 3 Seed Snackers                               1
Toe Nail Clip                                         1
Blackhead Eraser Scrub                                1
Name: product_name, Length: 39123, dtype: int64

In [43]:
train = combined
print(train.shape)
train.head()

(1384617, 7)


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,49302,1,1,Bulgarian Yogurt,120,16
1,816049,49302,7,1,Bulgarian Yogurt,120,16
2,1242203,49302,1,1,Bulgarian Yogurt,120,16
3,1383349,49302,11,1,Bulgarian Yogurt,120,16
4,1787378,49302,8,0,Bulgarian Yogurt,120,16


## How often is this product included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include the product?"

Let's wrangle!

In [47]:
product = 24852  # bananas
train[train['product_id'] == product]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
129688,226,24852,2,0,Banana,24,4
129689,473,24852,2,0,Banana,24,4
129690,878,24852,2,1,Banana,24,4
129691,1042,24852,1,1,Banana,24,4
129692,1139,24852,1,1,Banana,24,4
...,...,...,...,...,...,...,...
148409,3419531,24852,2,1,Banana,24,4
148410,3419542,24852,6,0,Banana,24,4
148411,3419629,24852,5,1,Banana,24,4
148412,3420088,24852,9,1,Banana,24,4


In [49]:
# 1.4% of the products ordered are (non-organic) bananas
(train['product_name'] == 'Banana').value_counts(normalize=True)

False    0.986476
True     0.013524
Name: product_name, dtype: float64

In [51]:
train.groupby('order_id').count()

,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
order_id,,,,,,
1,8,8,8,8,8,8
36,8,8,8,8,8,8
38,9,9,9,9,9,9
96,7,7,7,7,7,7
98,49,49,49,49,49,49
...,...,...,...,...,...,...
3421049,6,6,6,6,6,6
3421056,5,5,5,5,5,5
3421058,8,8,8,8,8,8


## Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

## How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought the product.

(And *none* of the orders from customers who have *never* bought the product.)

## What features can we engineer? We want to predict, will these customers reorder this product on their next order?